In [1]:
import subprocess
import pandas as pd
import os

In [2]:
from eval import calculate_nad, calculate_metrics
import joblib
import json
from adat.models.classification_model import LogisticRegressionOnTfIdf

In [3]:
from sklearn.model_selection import ParameterSampler
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from tqdm import tqdm
import numpy as np
import itertools
import time

In [27]:
#dataset = 'kaggle_transactions'
dataset = 'ag_news' 
#dataset = 'insurance'
#dataset = 'ai_academy_data'

In [28]:
base_dir = f'results/{dataset}'

In [29]:
result = []
for method in ['random']:
    base = os.path.join(base_dir, method)
    for folder in os.listdir(base):
        if folder != 'backup':
            args = json.load(open(os.path.join(base, folder, 'args.json')))
            metrics = json.load(open(os.path.join(base, folder, 'eval_metric.json')))
            args.update(metrics)
            result.append({k:args[k] for k in ['roc_auc_drop', 'accuracy_drop', 'probability_drop', 'WER', 'NAD',
                                                'beam_size', 'num_steps', 'std']})
result = pd.DataFrame(result)

In [30]:
print('random')
result[(result['beam_size'] == 30)&(result['num_steps'] == 30)].sort_values(by='NAD', ascending=False).head()

random


,roc_auc_drop,accuracy_drop,probability_drop,WER,NAD,beam_size,num_steps,std
0,0.277133,0.48,0.3385,5.14,0.143372,30,30,0.50
1,0.277133,0.48,0.3385,5.14,0.143372,30,30,0.05
2,0.277133,0.48,0.3385,5.14,0.143372,30,30,1.00
3,0.277133,0.48,0.3385,5.14,0.143372,30,30,0.20
4,0.277133,0.48,0.3385,5.14,0.143372,30,30,0.10


In [31]:
xs = os.listdir(f'results/{dataset}/hotflip')
x = np.sort(xs)[-1]
result = json.load(open(f'results/{dataset}/hotflip/{x}/eval_metric.json', 'r'))
result = pd.DataFrame([result])

In [32]:
print('hotflip')
result[['roc_auc_drop', 'accuracy_drop', 'probability_drop', 'WER', 'NAD']]

hotflip


,roc_auc_drop,accuracy_drop,probability_drop,WER,NAD
0,0.812639,0.81,0.686984,6.41,0.167634


In [33]:
result = []
for method in ['mcmc']:
    base = os.path.join(base_dir, method)
    for folder in os.listdir(base):
        if folder != 'backup':
            args = json.load(open(os.path.join(base, folder, 'args.json')))
            metrics = json.load(open(os.path.join(base, folder, 'eval_metric.json')))
            args.update(metrics)
            result.append({k:args[k] for k in ['roc_auc_drop', 'accuracy_drop', 'probability_drop', 'WER', 'NAD',
                                                'beam_size', 'num_steps', 'std', 'sigma_wer', 'sigma_class']})
result = pd.DataFrame(result)

In [34]:
print('mcmc')
result[(result['beam_size'] == 30)&(result['num_steps'] == 30)].sort_values(by='NAD', ascending=False).head()

mcmc


,roc_auc_drop,accuracy_drop,probability_drop,WER,NAD,beam_size,num_steps,std,sigma_wer,sigma_class
15,0.278004,0.48,0.337278,4.83,0.143372,30,30,0.20,1.00,0.75
10,0.277336,0.48,0.336949,4.87,0.143372,30,30,0.05,1.25,1.00
21,0.277336,0.48,0.336949,4.87,0.143372,30,30,0.20,0.75,0.75
20,0.277336,0.48,0.336949,4.87,0.143372,30,30,0.10,1.25,1.25
18,0.277336,0.48,0.336949,4.87,0.143372,30,30,0.50,1.25,1.00


In [35]:
result = []
for method in ['cascada']:
    base = os.path.join(base_dir, method)
    for folder in os.listdir(base):
        if folder != 'backup':
            args = json.load(open(os.path.join(base, folder, 'args.json')))
            metrics = json.load(open(os.path.join(base, folder, 'eval_metric.json')))
            args.update(metrics)
            result.append({k:args[k] for k in ['roc_auc_drop', 'accuracy_drop', 'probability_drop', 'WER', 'NAD',
                                               'levenshtein_weight', 'beam_size', 'max_steps', 'learning_rate']})
result = pd.DataFrame(result)

In [36]:
print('cascada')
result[(result['beam_size'] == 30)&(result['max_steps'] == 30)].sort_values(by='NAD', ascending=False).head()

cascada


,roc_auc_drop,accuracy_drop,probability_drop,WER,NAD,levenshtein_weight,beam_size,max_steps,learning_rate
33,0.566062,0.74,0.566426,5.50,0.171349,10.0,30,30,4.0
70,0.568571,0.75,0.570435,6.42,0.158386,0.1,30,30,4.0
65,0.557270,0.70,0.551676,5.61,0.157824,1.0,30,30,4.0
35,0.313292,0.45,0.354751,3.28,0.141968,0.1,30,30,2.0
54,0.308149,0.43,0.345154,3.28,0.133169,10.0,30,30,2.0
